.. _graph-visualization:

# Graph Visualization

## Overview

Toyplot now includes support for visualizing graphs - in the mathematical sense of vertices connected by edges - using the :meth:`toyplot.canvas.Canvas.graph` and :func:`toyplot.graph` functions.  As we will see, graph visualizations combine many of the aspects and properties of line plots (for drawing the edges), scatterplots (for drawing the vertices), and text (for drawing labels).

At a minimum, a graph can be specified as a collection of edges.  For example, consider a trivial social network:

In [1]:
sources = ["Tim", "Tim", "Fred", "Janet"]
targets = ["Fred", "Janet", "Janet", "Pam"]

... here, we have specified a sequence of source (start) vertices and target (end) vertices for each edge in the graph, which we can pass directly to Toyplot for rendering:

In [2]:
import toyplot
toyplot.graph(sources, targets, width=300);

Fred Janet Pam Tim Save as .csv

Simple as it is, Toyplot had to perform many steps to arrive at this figure:

* We specified a set of edges as input, and Toyplot *induced* a set of unique vertices from them.
* Used a *layout algorithm* to calculate coordinates for each vertex.
* Rendered the *vertices*.
* Rendered a set of *vertex labels*.
* Rendered an *edge* (line) between each pair of connected vertices.

We will examine each of these concepts in depth over the course of this guide.

## Inputs

At a minimum, you must specify the edges in a graph to create a visualization.  In the above example, we specified a sequence of edge sources and a sequence of edge targets.  We could also specify the edges as a numpy matrix (2D array) containing a column of sources and a column of targets:

In [3]:
import numpy
edges = numpy.array([["Tim", "Fred"], ["Tim", "Janet"], ["Fred", "Janet"], ["Janet", "Pam"]])
toyplot.graph(edges, width=300);

Fred Janet Pam Tim Save as .csv

In either case, Toyplot creates (*induces*) vertices using the edge source / target values.  Specifically, the source / target values are used as *vertex identifiers*, with a vertex created for each unique identifier.  Note that vertex identifiers don't have to be strings, as in the following example:

In [4]:
edges = numpy.array([[0, 1], [0, 2], [1, 2], [2, 3]])
toyplot.graph(edges, width=300);

0 1 2 3 Save as .csv

Inducing vertices from edge data is sufficient for many problems, but there may be occaisions when your graph contains disconnected vertices without any edge connections.  For this case, you may specify an optional collection of extra vertex identifiers to add to your graph:

In [5]:
extra_vertices=[10]
toyplot.graph(edges, extra_vertices, width=300);

0 1 2 3 10 Save as .csv

## Layout Algorithms

## Vertex Rendering

## Vertex Labels

## Edge Rendering